# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']
S. Rezaei  ->  S. Rezaei Kh.  |  ['S. Rezaei']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 68 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.15504


extracting tarball to tmp_2308.15504...

 done.


M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2308.16156


extracting tarball to tmp_2308.16156...

 done.
Retrieving document from  https://arxiv.org/e-print/2308.16165


extracting tarball to tmp_2308.16165...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.15504-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.15504) | **Planet formation throughout the Milky Way: Planet populations in the  context of Galactic chemical evolution**  |
|| J. Nielsen, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark> |
|*Appeared on*| *2023-08-31*|
|*Comments*| *21 pages, 16 figures, accepted in A&A*|
|**Abstract**| As stellar compositions evolve over time in the Milky Way, so will the resulting planet populations. In order to place planet formation in the context of Galactic chemical evolution, we make use of a large ($N = 5\,325$) stellar sample representing the thin and thick discs, defined chemically, and the halo, and we simulate planet formation by pebble accretion around these stars. We build a chemical model of their protoplanetary discs, taking into account the relevant chemical transitions between vapour and refractory minerals, in order to track the resulting compositions of formed planets. We find that the masses of our synthetic planets increase on average with increasing stellar metallicity [Fe/H] and that giant planets and super-Earths are most common around thin-disc ($\alpha$-poor) stars since these stars have an overall higher budget of solid particles. Giant planets are found to be very rare ($\lesssim$1\%) around thick-disc ($\alpha$-rich) stars and nearly non-existent around halo stars. This indicates that the planet population is more diverse for more metal-rich stars in the thin disc. Water-rich planets are less common around low-metallicity stars since their low metallicity prohibits efficient growth beyond the water ice line. If we allow water to oxidise iron in the protoplanetary disc, this results in decreasing core mass fractions with increasing [Fe/H]. Excluding iron oxidation from our condensation model instead results in higher core mass fractions, in better agreement with the core-mass fraction of Earth, that increase with increasing [Fe/H]. Our work demonstrates how the Galactic chemical evolution and stellar parameters, such as stellar mass and chemical composition, can shape the resulting planet population. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.15859-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.15859) | **On the source counts of VLBI-detected radio sources and the prospects of  all-sky surveys with current and next generation instruments**  |
|| <mark>S. Rezaei</mark>, J. P. McKean, A. T. Deller, J. F. Radcliffe |
|*Appeared on*| *2023-08-31*|
|*Comments*| **|
|**Abstract**| We present an analysis of the detection fraction and the number counts of radio sources imaged with Very Long Baseline Interferometry (VLBI) at 1.4 GHz as part of the mJIVE-20 survey. From a sample of 24,903 radio sources identified by FIRST, 4,965 are detected on VLBI-scales, giving an overall detection fraction of $19.9\pm2.9~$per cent. However, we find that the detection fraction falls from around 50 per cent at a peak surface brightness of $80~mJy~beam^{-1}$ in FIRST to around 8 per cent at the detection limit, which is likely dominated by the surface brightness sensitivity of the VLBI observations, with some contribution from a change in the radio source population. We also find that compactness at arcsec-scales is the dominant factor in determining whether a radio source is detected with VLBI, and that the median size of the VLBI-detected radio sources is 7.7 mas. After correcting for the survey completeness and effective sky area, we determine the slope of the differential number counts of VLBI-detected radio sources with flux densities $S_{\rm 1.4~GHz} > 1~mJy$ to be $\eta_{\rm VLBI} = -1.74\pm 0.02$, which is shallower than in the cases of the FIRST parent population ($\eta_{\rm FIRST} = -1.77\pm 0.02$) and for compact radio sources selected at higher frequencies ($\eta_{\rm JBF} = -2.06\pm 0.02$). From this, we find that all-sky ($3\pi~sr$) surveys with the EVN and the VLBA have the potential to detect $(7.2\pm0.9)\times10^{5}$ radio sources at mas-resolution, and that the density of compact radio sources is sufficient (5.3~deg$^{-2}$) for in-beam phase referencing with multiple sources (3.9 per primary beam) in the case of a hypothetical SKA-VLBI array. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.16155-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.16155) | **The Inhomogeneity Effect I: Inhomogeneous Surface and Atmosphere  Accelerate Planetary Cooling**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2023-08-31*|
|*Comments*| *26 pages, 8 figures, part one of a series of papers, accepted for publication in The Astrophysical Journal*|
|**Abstract**| We propose a general principle that under the radiative-convective equilibrium, the spatial and temporal variations in a planet's surface and atmosphere tend to increase its cooling. This principle is based on Jensen's inequality and the curvature of the response functions of surface temperature and outgoing cooling flux to changes in incoming stellar flux and atmospheric opacity. We use an analytical model to demonstrate that this principle holds for various planet types: (1) on an airless planet, the mean surface temperature is lower than its equilibrium temperature; (2) on terrestrial planets with atmospheres, the inhomogeneity of incoming stellar flux and atmospheric opacity reduces the mean surface temperature; (3) on giant planets, inhomogeneously distributed stellar flux and atmospheric opacity increase the outgoing infrared flux, cooling the interior. Although the inhomogeneity of visible opacity might sometimes heat the atmosphere, the effect is generally much smaller than the inhomogeneous cooling effect of infrared opacity. Compared with the homogeneous case, the mean surface temperature on inhomogeneous terrestrial planets can decrease by more than 20\%, and the internal heat flux on giant planets can increase by over an order of magnitude. Despite simplifications in our analytical framework, the effect of stellar flux inhomogeneity appears to be robust, while further research is needed to fully understand the effects of opacity inhomogeneity in more realistic situations. This principle impacts our understanding of planetary habitability and the evolution of giant planets using low-resolution and one-dimensional frameworks that may have previously overlooked the role of inhomogeneity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.16156-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.16156) | **The Inhomogeneity Effect II: Rotational and Orbital States Impact  Planetary Cooling**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2023-08-31*|
|*Comments*| *25 pages, 11 figures, part two of a series of papers, accepted for publication in The Astrophysical Journal*|
|**Abstract**| We generalize the theory of the inhomogeneity effect to enable comparison among different inhomogeneous planets. A metric of inhomogeneity based on the cumulative distribution function is applied to investigate the dependence of planetary cooling on previously overlooked parameters. The mean surface temperature of airless planets increases with rotational rate and surface thermal inertia, which bounds the value in the tidally locked configuration and the equilibrium temperature. Using an analytical model, we demonstrate that the internal heat flux of giant planets exhibits significant spatial variability, primarily emitted from the nightside and high-latitude regions acting as ``radiator fins." Given a horizontally uniform interior temperature in the convective zone, the outgoing internal flux increases up to several folds as the inhomogeneity of the incoming stellar flux increases. The enhancement decreases with increasing heat redistribution through planetary dynamics or rotation. The outgoing internal flux on rapidly rotating planets generally increases with planetary obliquity and orbital eccentricity. The radiative timescale and true anomaly of the vernal equinox also play significant roles. If the radiative timescale is long, the outgoing internal flux shows a slightly decreasing but nonlinear trend with obliquity. Our findings indicate that rotational and orbital states greatly influence the cooling of planets and impact the interior evolution of giant planets, particularly for tidally locked planets and planets with high eccentricity and obliquity (such as Uranus), as well as the spatial and temporal variations of their cooling fluxes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.16165-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.16165) | **The Inhomogeneity Effect III: Weather Impacts on the Heat Flow of Hot  Jupiters**  |
|| <mark>X. Zhang</mark>, C. Li, H. Ge, T. Le |
|*Appeared on*| *2023-08-31*|
|*Comments*| *36 pages, 13 figures, part three of a series of papers, accepted for publication in The Astrophysical Journal*|
|**Abstract**| The interior flux of a giant planet impacts atmospheric motion, and the atmosphere dictates the interior's cooling. Here we use a non-hydrostatic general circulation model (SNAP) coupled with a multi-stream multi-scattering radiative module (HARP) to simulate the weather impacts on the heat flow of hot Jupiters. We found that the vertical heat flux is primarily transported by convection in the lower atmosphere and regulated by dynamics and radiation in the overlying ``radiation-circulation" zone. The temperature inversion occurs on the dayside and reduces the upward radiative flux. The atmospheric dynamics relay the vertical heat transport until the radiation becomes efficient in the upper atmosphere. The cooling flux increases with atmospheric drag due to increased day-night contrast and spatial inhomogeneity. The temperature dependence of the infrared opacity greatly amplifies the opacity inhomogeneity. Although atmospheric circulation could transport heat downward in a narrow region above the radiative-convective boundary, the opacity inhomogeneity effect overcomes the dynamical effect and leads to a larger overall interior cooling than the local simulations with the same interior entropy and stellar flux. The enhancement depends critically on the equilibrium temperature, drag, and atmospheric opacity. In a strong-drag atmosphere hotter than 1600 K, a significant inhomogeneity effect in three-dimensional (3D) models can boost interior cooling several-fold compared to the 1D radiative-convection equilibrium models. This study confirms the analytical argument of the inhomogeneity effect in Zhang (2023ab). It highlights the importance of using 3D atmospheric models in understanding the inflation mechanisms of hot Jupiters and giant planet evolution in general. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.15504.md
    + _build/html/tmp_2308.15504/./Figures/MR_category_all.png
    + _build/html/tmp_2308.15504/./Figures/mgfe_mgh_feh.png
    + _build/html/tmp_2308.15504/./Figures/pebble_flux_st.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\maps}{M\&PS}$
$\newcommand{\feh}{\text{[Fe/H]}}$
$\newcommand{\abs}[1]{\left\lvert #1 \right\rvert}$
$\newcommand{\enstatite}{\text{MgSiO}_{3}}$
$\newcommand{\forsterite}{\text{Mg}_2\text{SiO}_4}$
$\newcommand{\water}{\text{H}_2\text{O}}$
$\newcommand{\fayalite}{\text{Fe}_2\text{SiO}_4}$
$\newcommand{\magnetite}{\text{Fe}_3\text{O}_4}$
$\newcommand{\St}{{\rm St}}$
$\newcommand{\AU}{{\rm AU}}$
$\newcommand{\pluseq}{\mathrel{{+}{=}}}$
$\newcommand{\minuseq}{\mathrel{{-}{=}}}$
$\newcommand{\gas}{{\rm (g)}}$
$\newcommand{\solid}{{\rm (s)}}$
$\newcommand{\vf}{v_{\rm f}}$
$\newcommand{\msun}{\ensuremath{\rm M_\odot}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\mb}{\textcolor{red}}$</div>



<div id="title">

# Planet formation throughout the Milky Way:

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.15504-b31b1b.svg)](https://arxiv.org/abs/2308.15504)<mark>Appeared on: 2023-08-31</mark> -  _21 pages, 16 figures, accepted in A&A_

</div>
<div id="authors">

J. Nielsen, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark>

</div>
<div id="abstract">

**Abstract:** As stellar compositions evolve over time in the Milky Way, so will the resulting planet populations. In order to place planet formation in the context of Galactic chemical evolution, we make use of a large ( $N = 5 325$ ) stellar sample representing the thin and thick discs, defined chemically, and the halo, and we simulate planet formation by pebble accretion around these stars. We build a chemical model of their protoplanetary discs, taking into account the relevant chemical transitions between vapour and refractory minerals, in order to track the resulting compositions of formed planets.We find that the masses of our synthetic planets increase on average with increasing stellar metallicity [ Fe/H ] and that giant planets and super-Earths are most common around thin-disc ( $\alpha$ -poor) stars since these stars have an overall higher budget of solid particles. Giant planets are found to be very rare ( $\lesssim$ 1 \% ) around thick-disc ( $\alpha$ -rich) stars and nearly non-existent around halo stars. This indicates that the planet population is more diverse for more metal-rich stars in the thin disc. Water-rich planets are less common around low-metallicity stars since their low metallicity prohibits efficient growth beyond the water ice line. If we allow water to oxidise iron in the protoplanetary disc, this results in decreasing core mass fractions with increasing [ Fe/H ] . Excluding iron oxidation from our condensation model instead results in higher core mass fractions, in better agreement with the core-mass fraction of Earth, that increase with increasing [ Fe/H ] . Our work demonstrates how the Galactic chemical evolution and stellar parameters, such as stellar mass and chemical composition, can shape the resulting planet population.

</div>

<div id="div_fig1">

<img src="tmp_2308.15504/./Figures/MR_category_all.png" alt="Fig8" width="100%"/>

**Figure 8. -** Resulting planet population for three solar-mass stars of varying composition. At low metallicity, it is difficult to form any planets beyond a few AU in the protoplanetary disc, so very few water-rich planets above 1 $M_\oplus$ form in that case. For solar metallicity and above (middle and bottom panels), we start to form water-rich planets up to around a few $M_\oplus$ far out in the disc. We note that due to our abundance scaling of the remaining elements, the middle panel has a slightly higher metallicity of 0.016 for solar [Fe/H] compared to the solar metallicity of 0.014. (*fig:MR_all*)

</div>
<div id="div_fig2">

<img src="tmp_2308.15504/./Figures/mgfe_mgh_feh.png" alt="Fig1" width="100%"/>

**Figure 1. -** Stellar abundances for our stellar sample where we have colour-coded the stars belonging to each population. **(a):**[Mg/Fe] as a function of [Fe/H]. **(b):**[Mg/H] as a function of [Fe/H] for the same stars as in the top panel. We also show the linear fit between [Mg/H] and [Fe/H] for each population which was used to determine all the other elements except for He. The fit slope ($k$) and intercept ($m$) for the full sample are shown in the bottom right. In both panels, the dashed black line shows the fit when considering the entire stellar sample. (*fig:mgfe_mgh_feh*)

</div>
<div id="div_fig3">

<img src="tmp_2308.15504/./Figures/pebble_flux_st.png" alt="Fig4" width="100%"/>

**Figure 4. -** The evolution of the dust over time in the protoplanetary disc. The different colours represent different distances to the star. **(a):** Pebble flux and flux ratio as a function of time around a star with solar mass and composition. The fragmentation velocity here was set to 2 m/s. The flux ratio starts at a moderately high value of $\sim$0.015 but decreases rapidly after $\sim10^5$ years. The sharp increase in pebble flux and flux ratio at 1 Myr is caused by the $CO_2$ iceline which is traveling inwards due to the decreasing luminosity of the star. **(b):** Stokes number as a function of time for the same distances as in the top panel. The Stokes number increases fast in the inner region of the disc, growing to 0.02 before 10$^3$ years. The size of the dust is mostly regulated by fragmentation either due to turbulence or differential radial drift. Only in the outermost regions of the disc is the growth of the dust slow enough for the Stokes number to be drift-limited. (*fig:pebble_flux*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.15504"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

258  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
